<h1> FOOFing OMEGA Data </h1>

This notebook runs FOOF on MEG-PSD data.
- Option to Parallelize with Pool from Multiprocessing
- Must Specify frequency range desired for FOOF
- Saves FOOF results in pickled object

NOTES:
- For reasons unknown, parallel implementation with Pool does not currently work. 

    -It used to, but now hangs when trying to parallelize foof. 
    
- Parallel with ipyparallel is currently working!

In [1]:
# Import general code from custom module om
from __future__ import print_function

import sys
sys.path.append('/Users/thomasdonoghue/Documents/GitCode/omegamappin/')
from om.gen import *

# Import required packages
import numpy as np

# Set plots inline
%matplotlib inline

In [2]:
# Get database and path details from OMDB class
db = OMDB('HCP')

In [3]:
# Check available subject numbers for PSD files
sub_nums = db.check_dat_files('PSD')


Number of Subjects available: 60

Subject numbers with FOOF data available: 
[100307, 102816, 105923, 106521, 109123, 111514, 112920, 113922, 116524, 116726, 140117, 146129, 153732, 154532, 156334, 158136, 162026, 162935, 164636, 166438, 172029, 174841, 175237, 175540, 181232, 185442, 187547, 189349, 191033, 191437, 191841, 192641, 195041, 198653, 204521, 205119, 212318, 212823, 214524, 221319, 223929, 233326, 248339, 250427, 255639, 257845, 283543, 293748, 352132, 352738, 353740, 358144, 406836, 433839, 512835, 555348, 559053, 568963, 581450, 599671]



In [4]:
# Check which PSD files haven't been FOOFed
psd_files, _ = db.check_dat_files('PSD', verbose=False)
foof_files, _ = db.check_dat_files('foof', verbose=False)

not_foofed = []
for f in psd_files:
    if f not in foof_files:
        not_foofed.append(f)

print(str(len(not_foofed)), 'Subject Files not yet FOOFed: ')
print(not_foofed)

9 Subject Files not yet FOOFed: 
[358144, 406836, 433839, 512835, 555348, 559053, 568963, 581450, 599671]


## CHECK PSD DATA

In [5]:
# Load MEG Data
subj = 352132
psd, freqs = load_meg_psds(db.dat_source, db.psd_path, subj)

In [6]:
# Check data - get number of PSDs and frequency resolution
[nPSDs, nFreqs] = np.shape(psd)
freq_res = np.mean(np.diff(freqs))
f_min = freqs.min()
f_max = freqs.max()
print('Number of PSDs is: ', nPSDs)
print('Frequency Resolution is: ', freq_res)
print('Frequency range is: ', f_min, ' to ', f_max)

Number of PSDs is:  7500
Frequency Resolution is:  0.488281249999
Frequency range is:  0.0  to  499.999999999


In [42]:
# Extract PSD range of interest
psd_ext, freqs_ext = extract_psd(psd, freqs, f_low=3, f_high=40)

In [43]:
# Set up PSD as a list of 2-D np arrays
psd_list = list(psd_ext)
for i in range(0, nPSDs):
    psd_list[i] = np.reshape(psd_list[i], [len(freqs_ext), 1])

In [77]:
# Run FOOF
foof_results = meg_foof(psd_ext[nan_vertices, :], freqs_ext, min_p=0.1, freq_res=freq_res)

# Save out results
#save_path = '/Users/thomasdonoghue/Documents/Research/1-Projects/OMEGA/2-Data/MEG/3-FOOF/Colin/pickle'
#save_foof_pickle(foof_results, save_path, subj)

In [ ]:
# Check for NaNs in BW
centers, _, bws, _, _ = extract_foof_pickle(foof_results)
sum(np.isnan(bws))

### Save out data for NaN tests

In [44]:
# Set data to save out
psd_ext = psd_ext[nan_vertices]
freq_res = np.mean(np.diff(freqs))
min_p = 0.1

In [45]:
# Save data out to npz file
np.savez('NaN_PSDs_369737.npz', psd_ext=psd_ext, freqs_ext=freqs_ext, freq_res=freq_res, min_p=min_p)

## Check PSDs

In [ ]:
import matplotlib.pyplot as plt

plt.plot(freqs_ext[:], psd_ext[2500, :])

### Convert pickle files (of saved FOOF results) to csv

In [ ]:
## To convert a single file

# Set subject number to convert
subj = 559176

# Get database object - set which database to use
db = OMDB('OMEGA')

# Load data from pickle file and resave as csv file
dat = load_foof_pickle(db.foof_path, subj)
#save_foof_csv(dat, db.foof_path, subj)

In [ ]:
print(len(dat))
print(type(dat))
print(type(dat[0]))

In [ ]:
## To convert all files
# NOTE: This will not check which files are already converted to csv, and will overwrite pre-existing csv files

# Get database object - set which database files to check
db = OMDB('HCP')

# Get list of foofed pickle files
foofed_files = db.check_files('foof', save_type='pickle', verbose=False)

# Loop through each subj, load foof from pickle, and save out as a csv
for subj in sub_nums:
    
    # Load subj foof data, from pickle file
    dat = load_foof_pickle(db.foof_path, subj)
    
    # Save out as csv file
    save_foof_csv(dat, db.foof_path, subj)

## Check FOOF Files

In [6]:
# Check available subject numbers for FOOF files
sub_nums = db.check_dat_files('foof')


Number of Subjects available: 25

Subject numbers with FOOF data available: 
[111802, 124340, 195121, 215187, 218131, 220216, 245518, 265922, 320548, 339964, 369737, 386068, 390845, 403435, 405716, 431088, 504997, 548027, 559176, 604600, 704718, 704814, 719634, 756575, 978341]



## Check FOOF Results

In [30]:
# Load from pickled foof files, and pull out data
results = load_foof_pickle(db.foof_path , 369737)
centers, powers, bws, _, n_psds = extract_foof_pickle(results)

In [31]:
# Check for nan values in bandwidths
bw_nan_bool = np.isnan(bws)
n_nans = sum(bw_nan_bool)

print(n_nans)

[109   0   0   0   0   0   0   0]


In [32]:
# Get indices of NaN values
nan_vertices, nan_oscs = np.where(bw_nan_bool == True)

# Check what data becomes NaN
ind = 4
print(centers[nan_vertices[ind], nan_oscs[ind]])
print(bws[nan_vertices[ind], nan_oscs[ind]])

3.41796875
nan


## EXTRA STUFF


To check / finish

In [ ]:
# EXTRA: CHECK FOOF

print("model: f {0}, f_sig {1}, chi {2}".format(
        np.round(foof.centers_, decimals=1),
        np.round(foof.stdevs_, decimals=1),
        np.round(foof.chi_, decimals=1))
     )

plt.subplot(211)
plt.plot(freqs_ext, psd_example[:,0]*10000000000000000000000, label='Data')
plt.plot(*syn.mfonef(mk=foof.powers_, 
                     mf=foof.centers_, 
                     mf_sig=foof.stdevs_, 
                     f0=3, 
                     fmax=50, 
                     chi=foof.chi_, 
                     res=freq_res), 
         label='Model'
        )
plt.xlabel("F (Hz)")
plt.ylabel("PSD")
plt.yscale('log')
plt.xscale('log')
plt.legend()